<a href="https://colab.research.google.com/github/jrtedeschi/data_processing_colab/blob/master/Data_Processing_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Processing Pipeline**
## To run everything automatically just click **Runtime -> Run All on the top left corner of the screen**
### This Google Colab notebook has the purpose of processing .csv files. It will perform data Extraction, Transformation and Loading (ETL Pipeline)  the resulting data into a processed file to serve as feed for a Data Visualization tool.

*First, we begin importing the necessary libraries from Python*

In [0]:
from google.colab import drive # this one is responsible for accessing your Drive Folders
import os # this is to deal with de Operational System
import glob # dealing with files
import time # timing
import re # regular expressions and text analysing
import pandas as pd # data processing
import numpy as np # numerical operations
import tqdm # code profiling
import matplotlib.pyplot as plt # graphical library

#graphical
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20,viridis

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application


# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

*Now we are going to Mount your Drive (or attach your folders to this virtual machine) to this notebook*

Follow these Steps:

1.   Click the URL that will appear
2.   Choose your main Drive account
3.   Sign and allow Google Drive File Stream
4.   Copy and paste the authorization code and hit "Enter"

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


*All your Drive folders are available now, so we are going to check if the files you want are here*

***Be sure that the path and filename pattern are the ones you want***

In [3]:
all_files = glob.glob("drive/My Drive/Profits/Helium10_ProductPerformance_*.csv") # listing the Product Performance CSV files

for file_name in all_files:
  print(file_name)
  

drive/My Drive/Profits/Helium10_ProductPerformance_2020-04-05_2020-04-11.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-03-29_2020-04-04.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-03-22_2020-03-28.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-03-15_2020-03-21.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-03-08_2020-03-14.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-03-01_2020-03-07.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-02-23_2020-02-29.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-02-16_2020-02-22.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-02-09_2020-02-15.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-02-02_2020-02-08.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-01-26_2020-02-01.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-01-19_2020-01-25.csv
drive/My Drive/Profits/Helium10_ProductPerformance_2020-01-12_2020-01-18.csv

*Great, let's process them*

In [4]:
dataframes = list() # Creating an empty DataFrames list
date_columns = ["Start Date","End Date"]

for file_name in tqdm.tqdm(all_files): # Processing files into DataFrames
    df = pd.read_csv(file_name)
    for date_column in date_columns:
      df[date_column] = pd.to_datetime(df[date_column])
    df["Year_Week"] = df["Start Date"].dt.strftime('%Y%V')
    df["Year"] = df["Start Date"].dt.year
    df["Week"] = df["Start Date"].dt.week
    dataframes.append(df)

100%|██████████| 20/20 [00:04<00:00,  4.12it/s]


In [5]:
df = pd.concat(dataframes,ignore_index=True) # concatenation of all dataframes into one
df.head()

,Title,Asin,Seller Sku,Gross,Cost,Net,Margin %,ROI %,Promos,Refunds,Units,Marketplace,Start Date,End Date,Year_Week,Year,Week
0,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B07DGVMQ98,FF-4SSN-JQFC,33.99,28.74,5.25,15,117,0.0,0,1.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
1,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B013FFGQ6S,JE-3DVQ-TEGA,72.48,36.79,35.69,49,397,0.0,0,2.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
2,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B076CT7DZ6,7W-MXQL-TXFS,72.52,62.04,10.48,14,117,1.0,1,2.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
3,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B07DGKHCNQ,CS-TEUO-M64U,253.21,119.47,133.74,53,426,0.0,0,7.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
4,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B013FFGQ54,PB-JWUE-HYNA,73.01,47.71,25.30,35,282,0.0,0,2.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14


*Let's get the products list*

In [6]:
products_path = "drive/My Drive/Profits/Products.xlsx" 

products = pd.read_excel(products_path)
products.head() # prints out the first five rows

,ASIN,Title
0,B07WMWG16Z,4 Inch Belt
1,B07WFWG9ZP,4 Inch Belt
2,B07WFWFLWB,4 Inch Belt
3,B07WHZNCTM,4 Inch Belt
4,B07WHZNTY4,4 Inch Belt


*Filtering the DataFrame with ASINs we want to aggregate*

In [7]:
filtered_df = df.loc[df["Asin"].isin(products["ASIN"])]
filtered_df.head()

,Title,Asin,Seller Sku,Gross,Cost,Net,Margin %,ROI %,Promos,Refunds,Units,Marketplace,Start Date,End Date,Year_Week,Year,Week
0,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B07DGVMQ98,FF-4SSN-JQFC,33.99,28.74,5.25,15,117,0.0,0,1.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
1,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B013FFGQ6S,JE-3DVQ-TEGA,72.48,36.79,35.69,49,397,0.0,0,2.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
2,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B076CT7DZ6,7W-MXQL-TXFS,72.52,62.04,10.48,14,117,1.0,1,2.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
3,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B07DGKHCNQ,CS-TEUO-M64U,253.21,119.47,133.74,53,426,0.0,0,7.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14
4,"Fire Team Fit Weightlifting Belt, Weight Belt,...",B013FFGQ54,PB-JWUE-HYNA,73.01,47.71,25.30,35,282,0.0,0,2.0,www.amazon.com,2020-04-05,2020-04-11,202014,2020,14


*Here the cumulative Net amount per week of the year is calculated, by title and ASIN in case you wanto see performance by ASIN later.*

In [8]:
aggregated_numbers = filtered_df.groupby(["Asin","Year_Week","End Date"])["Net"].sum().reset_index()

final_df = aggregated_numbers.merge(products,how= "left",left_on= "Asin" ,right_on="ASIN")
final_df = final_df.drop(columns="Asin").sort_values(by= ["ASIN","Year_Week"])

left_df = final_df.groupby(["ASIN","Year_Week","End Date","Title"])["Net"].sum().groupby(level=[0]).cumsum()\
                                                     .reset_index().rename(columns={"Net": "Cumulative Net"})
                                                                                                        
final_df = final_df.merge(left_df["Cumulative Net"],left_index=True,right_index=True,how="inner")
final_df

,Year_Week,End Date,Net,ASIN,Title,Cumulative Net
0,201948,2019-12-07,307.92,B013FEODYQ,6 Inch Belt,307.92
1,201949,2019-12-14,839.66,B013FEODYQ,6 Inch Belt,1147.58
2,201950,2019-12-21,908.44,B013FEODYQ,6 Inch Belt,2056.02
3,201951,2019-12-28,191.39,B013FEODYQ,6 Inch Belt,2247.41
4,201952,2020-01-04,345.17,B013FEODYQ,6 Inch Belt,2592.58
...,...,...,...,...,...,...
1934,202011,2020-03-21,4.23,B07Y9KT39P,Dip Belt,778.94
1935,202012,2020-03-28,-37.33,B07Y9KT39P,Dip Belt,741.61
1936,202013,2020-04-04,31.12,B07Y9KT39P,Dip Belt,772.73
1937,202014,2020-04-11,92.74,B07Y9KT39P,Dip Belt,865.47


*Now the spreadsheet is created and the data is prepared for the graphing/plotting section*

Check your Drive/Profits folder, the file should be there by the name **Title_Performance_Weekly**

In [0]:
sliced_dfs = [final_df.loc[final_df["Title"] == title].groupby(["Year_Week","End Date","Title"])["Cumulative Net"].sum().reset_index() for title in final_df['Title'].unique()]
to_spreadsheet = pd.concat(sliced_dfs,ignore_index=True)
to_spreadsheet.to_excel("drive/My Drive/Profits/Title_Performance_Weekly.xlsx",sheet_name="Performance Weekly",index=False) #check your profits folder, the file might be there

*Data processing step*

In [0]:
sliced_dfs = [final_df.loc[final_df["Title"] == title].groupby(["Year_Week","End Date","Title"])["Cumulative Net"].sum().reset_index() for title in final_df['Title'].unique()]
color_range = viridis(len(sliced_dfs))
titles_len = len(sliced_dfs)
dates = [list(df["End Date"]) for df in sliced_dfs]
values = [list(df["Cumulative Net"]) for df in sliced_dfs]
titles = list(final_df['Title'].unique())

data = {'xs': dates,
        'ys': values,
        'labels': titles,
        'colors' : color_range
        }
source = ColumnDataSource(data)

*Graph styling step*

In [0]:
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '16pt'
        p.title.text_font = 'helvetica'

        # Axis titles
        p.xaxis.axis_label_text_font = 'helvetica'
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font = 'helvetica'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

# Graph section

Here is the graph, you can save the image by clicking on the "Save" icon, by the right side the graph.


In [14]:

p = figure(plot_width=800, 
           plot_height=600,
           x_axis_type='datetime',
           y_axis_type = 'linear',
           title = 'Weekly Cumulative Net amount by Title',
           x_axis_label = 'Date', y_axis_label = 'Cumulative Net Amount'
           )

style(p)
for i in range(0,titles_len):
  xs = data['xs'][i]
  ys = data['ys'][i]
  labels = data['labels'][i]
  colors = data['colors'][i]
  p.line(x=xs, y=ys, line_width=3, color=colors, alpha=0.8, legend_label=labels)

hover = HoverTool(tooltips=[ 
                            ('Value', '@y{0,0}')
                            ],
                          mode='mouse',
                  formatters={'$x' : 'datetime'}
                  )
p.add_tools(hover)
p.legend.location = "top_left"
p.legend.click_policy="hide"


show(p)